In [2]:
import os
import torch
import torch.nn as nn
from torch import cuda
from torchvision import models
import albumentations as A
from modules.model import create_model

import numpy as np
import pandas as pd
from tqdm import tqdm
from argparse import ArgumentParser

from dataloader import CustomDataLoader, do_transform, collate_fn
from utils.utils import label_accuracy_score, add_hist, set_seed
from modules.losses import create_criterion


In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
num_workers = 4
batch_size = 8
resize = 256

val_json_path = '/opt/ml/input/data/seed21/val_0.json'
data_dir = '/opt/ml/input/data'
model_dir = '/opt/ml/trained_models/Final_fold_0'   # = save_dir
use_model = 'mvtb4_unet'
use_loss = 'combo'

In [4]:
def validation(model, data_loader, criterion, device):
    model.eval()
    category_names = ['Background', 'General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing']

    with torch.no_grad():
        n_class = 11
        total_loss = 0
        cnt = 0
        hist = np.zeros((n_class, n_class))

        for step, (images, masks, image_infos) in enumerate(tqdm(data_loader)):
            images = torch.stack(images)       
            masks = torch.stack(masks).long()
            images, masks = images.to(device), masks.to(device)            
            
            # device 할당
            model = model.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, masks)
            total_loss += loss
            cnt += 1
            outputs = torch.argmax(outputs, dim=1).detach().cpu().numpy()
            masks = masks.detach().cpu().numpy()

            hist = add_hist(hist, masks, outputs, n_class=n_class)

        acc, acc_cls, mIoU, fwavacc, IoU = label_accuracy_score(hist)
        IoU_by_class = [{classes : round(IoU,4)} for IoU, classes in zip(IoU , category_names)]
        avrg_loss = total_loss / cnt
        print(f'Validation. Average Loss: {round(avrg_loss.item(), 4)}, Accuracy : {round(acc, 4)}, mIoU: {round(mIoU, 4)}')
        print(f'IoU by class : {IoU_by_class}')
        
    return avrg_loss, mIoU, IoU_by_class

In [5]:
def greedy_soup(model_file_names, use_model=use_model, use_loss=use_loss, 
                data_dir=data_dir, model_dir=model_dir, val_json_path=val_json_path,
                device=device, num_workers=num_workers, batch_size=batch_size, resize=resize):
    # data loader
    val_dataset = CustomDataLoader(data_dir=val_json_path,
                                   mode='val',
                                   transform=do_transform(mode='val'),
                                   data_path=data_dir)                                   
    val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                             batch_size=batch_size,
                                             shuffle=False,
                                             num_workers=num_workers,
                                             collate_fn=collate_fn)

    n_class = 11
    criterion = create_criterion(use_loss)
    FLAG_First = True
    stacked_files = []

    # load models
    for model_file_name in model_file_names:
        print(model_file_name)
        model = create_model(use_model)
        checkpoint = torch.load(os.path.join(model_dir, model_file_name), map_location=device)
        state_dict = checkpoint.state_dict()
        model.load_state_dict(state_dict)
        model = model.to(device)

        if FLAG_First:
            model_soup = model
            #avrg_loss_soup, mIoU_soup, IoU_by_class_soup = validation(model, val_loader, criterion, device)
            FLAG_First = False
            stacked_files.append(model_file_name)
            continue
        
        else:
            beta = len(stacked_files)/(len(stacked_files) + 1)
            params_soup = model_soup.named_parameters()
            params_temp = model.named_parameters()

            dict_params_temp = dict(params_temp)
            print(beta)
            print(dict_params_temp['model.encoder.patch_embed1.proj.weight'][0, 0, 0])
            for name_soup, param_soup in params_soup:
                print(param_soup.data)
                if name_soup in dict_params_temp:
                    dict_params_temp[name_soup].data.copy_(beta*param_soup.data + (1-beta)*dict_params_temp[name_soup].data)
                else: assert False, 'No param named ' + name_soup
            print(dict_params_temp['model.encoder.patch_embed1.proj.weight'][0, 0, 0])
            #model_soup_temp = 
            
        assert False





In [6]:
# sort models (use only topK-saved dir)
file_list = [f for f in os.listdir(model_dir) if f.endswith('.pt')]
file_list.sort(key=lambda x: float(x[-9:-3]), reverse=True)
print(file_list)
assert False
greedy_soup(file_list)

#test_with_maps(model_paths, use_models)

['ep89_0.7374.pt', 'ep88_0.7366.pt', 'ep87_0.7363.pt', 'ep90_0.7363.pt', 'ep84_0.7352.pt', 'ep86_0.7329.pt', 'ep85_0.7327.pt', 'ep79_0.7306.pt', 'ep81_0.7242.pt', 'ep77_0.7232.pt', 'ep113_0.7191.pt', 'ep114_0.7181.pt', 'ep130_0.7177.pt', 'ep118_0.7175.pt', 'ep83_0.7174.pt']


AssertionError: 

In [ ]:


    size = resize
    transform = A.Compose([A.Resize(size, size)])
    print('\nStart prediction.')
    
    # Predict
    file_name_list = []
    preds_array = np.empty((0, size*size), dtype=np.long)
    
    with torch.no_grad():
        hist = np.zeros((n_class, n_class))
        for step, (imgs, masks, image_infos) in enumerate(tqdm(val_loader)):
            # inference (512 x 512)
            outs = []
            for model in models:
                outs.append(model(torch.stack(imgs).to(device)))     # model(torch.stack(imgs).to(device)): torch.Size([N_batch, 11, 512, 512])
            outs = torch.stack(outs)
            
            
            
            outs = torch.mean(outs, axis=0)
            oms = torch.argmax(outs.squeeze(), dim=1).detach().cpu().numpy()
            
            # resize (256 x 256)
            temp_mask = []
            for img, mask in zip(np.stack(imgs), oms):
                transformed = transform(image=img, mask=mask)
                mask = transformed['mask']
                temp_mask.append(mask)
                
            oms = np.array(temp_mask)
            
            oms = oms.reshape([oms.shape[0], size*size]).astype(int)
            preds_array = np.vstack((preds_array, oms))
            
            file_name_list.append([i['file_name'] for i in image_infos])

    print("End prediction.")
    file_names = [y for x in file_name_list for y in x]
    
    print("Start Inference")
    # sample_submisson.csv 열기
    submission = pd.read_csv('/opt/ml/input/code/submission/sample_submission.csv', index_col=None)

    # PredictionString 대입
    for file_name, string in zip(file_names, preds_array):
        submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                    ignore_index=True)

    # submission.csv로 저장
    submission.to_csv(os.path.join(save_dir, 'submission.csv'), index=False)
    print("End Inference")